In [1]:
!pip -q install transformers==4.17
!pip -q install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 40.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.0 MB/s eta 0:00:00


In [2]:
from transformers import Trainer, TrainingArguments, RobertaTokenizer, RobertaModel, RobertaForSequenceClassification, AutoTokenizer
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time

In [3]:
splits = {'python_train': 'data/python_train-00000-of-00001.parquet', 'python_test': 'data/python_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_test"])

display(train_df.head())
display(test_df.head())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,1,AccessMixin,functionality.,0,functionality. | AccessMixin,"[0, 0, 0, 0, 1]"
1,8,Atomic,"when it s used as a decorator, call wraps the ...",0,"when it s used as a decorator, call wraps the ...","[1, 0, 0, 0, 0]"
2,10,Atomic,"when it s used as a context manager, enter cre...",0,"when it s used as a context manager, enter cre...","[1, 0, 0, 0, 0]"
3,12,Atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,Atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,AccessMixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,AmbiguityError,more than one migration matches a name prefix.,1,more than one migration matches a name prefix....,"[0, 0, 0, 0, 1]"
2,3,AppConfigStub,stub of an appconfig.,1,stub of an appconfig. | AppConfigStub,"[0, 0, 0, 0, 1]"
3,4,AppConfigStub,only provides a label and a dict of models.,1,only provides a label and a dict of models. | ...,"[0, 1, 0, 0, 0]"
4,5,Archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


In [4]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,1,accessmixin,functionality.,0,functionality | accessmixin,"[0, 0, 0, 0, 1]"
1,8,atomic,"when it s used as a decorator, call wraps the ...",0,when it s used as a decorator call wraps the e...,"[1, 0, 0, 0, 0]"
2,10,atomic,"when it s used as a context manager, enter cre...",0,when it s used as a context manager enter crea...,"[1, 0, 0, 0, 0]"
3,12,atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,accessmixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,ambiguityerror,more than one migration matches a name prefix.,1,more than one migration matches a name prefix ...,"[0, 0, 0, 0, 1]"
2,3,appconfigstub,stub of an appconfig.,1,stub of an appconfig | appconfigstub,"[0, 0, 0, 0, 1]"
3,4,appconfigstub,only provides a label and a dict of models.,1,only provides a label and a dict of models | a...,"[0, 1, 0, 0, 0]"
4,5,archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


In [5]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [6]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [7]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [8]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**model_inputs) 
        logits = outputs.get("logits")

        if logits is None:
            raise ValueError("Logits are missing from the model output.") 

        # Use BCEWithLogitsLoss for multi-label classification
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [9]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)

    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = (preds == labels).mean()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [10]:
def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        warmup_steps=10,
        weight_decay=weight_decay,
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        gradient_accumulation_steps=2,
        report_to="none",
    )

    trainer = OneHotTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

In [11]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Best trial:")
print(study.best_trial)

[I 2024-12-03 17:37:06,709] A new study created in memory with name: no-name-acd808ba-c1ce-4ed7-af17-e69c95df869d
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=Tru

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.480986,0.787798,0.242640,0.038741,0.066814
1,No log,0.403726,0.843501,0.609820,0.377724,0.454801
2,No log,0.382152,0.851989,0.574094,0.460048,0.508127


***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in la

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-12-03 17:40:30,554] Trial 0 finished with value: 0.5081273306161127 and parameters: {'learning_rate': 3.402796222957594e-05, 'batch_size': 16, 'num_train_epochs': 3, 'weight_decay': 0.011024836319189686}. Best is trial 0 with value: 0.5081273306161127.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This featu

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.381324,0.849867,0.647859,0.537530,0.557665
1,No log,0.349867,0.862069,0.675904,0.542373,0.590148
2,0.326100,0.356325,0.858886,0.637829,0.578692,0.600977
3,0.326100,0.351538,0.857825,0.639149,0.595642,0.616531
4,0.326100,0.350883,0.854642,0.635721,0.593220,0.611737
5,0.262600,0.346484,0.862069,0.645939,0.600484,0.621180
6,0.262600,0.346325,0.859416,0.691321,0.600484,0.625297
7,0.222000,0.352535,0.859947,0.733025,0.612591,0.624725
8,0.222000,0.350289,0.860477,0.701647,0.612591,0.629043


...

[I 2024-12-03 18:32:09,268] Trial 8 finished with value: 0.7204038732487624 and parameters: {'learning_rate': 2.9129784853795005e-06, 'batch_size': 16, 'num_train_epochs': 4, 'weight_decay': 0.011606996728274414}. Best is trial 6 with value: 0.7273703652729121.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: 

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.537820,0.875862,0.726040,0.709443,0.717140
1,No log,0.533898,0.877454,0.734875,0.711864,0.721565
2,No log,0.531153,0.877454,0.736583,0.707022,0.720132


***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
***** Running Evaluation *****
  Num examples = 377
  Batch size = 32
***** Running Evaluation *****
  Num examples = 377
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 377
  Batch size = 32


[I 2024-12-03 18:35:27,423] Trial 9 finished with value: 0.7201320505873573 and parameters: {'learning_rate': 1.03416016002016e-06, 'batch_size': 16, 'num_train_epochs': 3, 'weight_decay': 0.0014317096033832073}. Best is trial 6 with value: 0.7273703652729121.


Best trial:
FrozenTrial(number=6, state=TrialState.COMPLETE, values=[0.7273703652729121], datetime_start=datetime.datetime(2024, 12, 3, 18, 20, 41, 729707), datetime_complete=datetime.datetime(2024, 12, 3, 18, 24, 13, 967669), params={'learning_rate': 6.369414513935845e-06, 'batch_size': 8, 'num_train_epochs': 3, 'weight_decay': 0.0014542879427658277}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-06, step=None), 'batch_size': CategoricalDistribution(choices=(4, 8, 16)), 'num_train_epochs': IntDistribution(high=10, log=False, low=3, step=1), 'weight_decay': FloatDistribution(high=0.1, log=True, low=0.0001, step=None)}, trial_id=6, value=None)


In [12]:
best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=best_hyperparams["num_train_epochs"],
    per_device_train_batch_size=best_hyperparams["batch_size"],
    per_device_eval_batch_size=best_hyperparams["batch_size"] * 2,
    learning_rate=best_hyperparams["learning_rate"],
    weight_decay=best_hyperparams["weight_decay"],
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time
print("<<=================Training Time================>>")
print(f"Training time: {training_time:.2f} seconds")
print("<<==============================================>>")

PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1507
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 282


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.541811,0.879045,0.730809,0.699758,0.713415
1,No log,0.546354,0.872679,0.731747,0.694915,0.710346
2,No log,0.534776,0.877984,0.728530,0.711864,0.719443


***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




<<=================Training Time================>>
Training time: 207.05 seconds
<<==============================================>>


In [13]:
eval_results = trainer.evaluate()
print(eval_results)

***** Running Evaluation *****
  Num examples = 377
  Batch size = 16


{'eval_loss': 0.534775972366333, 'eval_accuracy': 0.8779840848806366, 'eval_precision': 0.7285300731695988, 'eval_recall': 0.711864406779661, 'eval_f1': 0.7194425299788136, 'eval_runtime': 4.9708, 'eval_samples_per_second': 75.844, 'eval_steps_per_second': 4.828, 'epoch': 2.99}


In [14]:
!mkdir 'best_model_roberta-large_python'
!mkdir 'best_model_roberta-large_python_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_roberta-large_python')
tokenizer.save_pretrained('./best_model_roberta-large_python_tokenizer')

# Load model and tokenizer
from transformers import RobertaForSequenceClassification, RobertaTokenizer

model = RobertaForSequenceClassification.from_pretrained('./best_model_roberta-large_python')
tokenizer = RobertaTokenizer.from_pretrained('./best_model_roberta-large_python_tokenizer')

Saving model checkpoint to ./best_model_roberta-large_python
Configuration saved in ./best_model_roberta-large_python/config.json
Model weights saved in ./best_model_roberta-large_python/pytorch_model.bin
tokenizer config file saved in ./best_model_roberta-large_python_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_roberta-large_python_tokenizer/special_tokens_map.json
loading configuration file ./best_model_roberta-large_python/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,


## Model Testing

In [15]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [16]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

print(test_label)

[array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1])
 array([0, 1, 0, 0, 0]) array([0, 0, 1, 0, 1]) array([0, 0, 0, 0, 1])
 array([0, 0, 0, 0, 1]) array([1, 0, 0, 1, 0]) array([1, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0]) array([1, 0, 0, 0, 0]) array([1, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0]) array([0, 0, 0, 0, 1]) array([0, 0, 0, 1, 0])
 array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0])
 array([0, 0, 0, 0, 1]) array([0, 0, 0, 1, 0]) array([1, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0]) array([0, 0, 1, 0, 0]) array([0, 0, 0, 0, 1])
 array([1, 0, 0, 0, 0]) array([1, 0, 0, 0, 0]) array([0, 1, 0, 0, 0])
 array([0, 1, 0, 0, 0]) array([0, 1, 0, 0, 0]) array([0, 1, 0, 0, 0])
 array([0, 1, 0, 0, 0]) array([0, 1, 0, 0, 0]) array([0, 0, 0, 1, 0])
 array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0])
 array([0, 0, 1, 0, 0]) array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1])
 array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1]) array([0, 0, 1, 0, 0])
 array([0, 0, 0, 0, 

In [17]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

start_time = time.time()

predictions = trainer.predict(test_dataset)

end_time = time.time()
inference_time = end_time - start_time

metrics = compute_metrics(predictions)

print("<<==============================================>>")
print("<<=================Evaluation Metrics================>>")
print("Evaluation Metrics:", metrics)
print("<<==============================================>>")
print("<<=================Inference Time================>>")
print(f"Inference Time: {inference_time:.2f} seconds")
print("<<==============================================>>")

***** Running Prediction *****
  Num examples = 406
  Batch size = 16


<<==============================================>>
<<=================Evaluation Metrics================>>
Evaluation Metrics: {'accuracy': 0.8379310344827586, 'precision': 0.6285414471170903, 'recall': 0.6055045871559633, 'f1': 0.6166531407624688}
<<==============================================>>
<<=================Inference Time================>>
Inference Time: 5.21 seconds
<<==============================================>>


In [18]:
!zip -r best_model_roberta-large_python.zip './best_model_roberta-large_python'
!zip -r best_model_roberta-large_python_tokenizer.zip './best_model_roberta-large_python_tokenizer'

  adding: best_model_roberta-large_python/ (stored 0%)
  adding: best_model_roberta-large_python/pytorch_model.bin (deflated 15%)
  adding: best_model_roberta-large_python/training_args.bin (deflated 51%)
  adding: best_model_roberta-large_python/config.json (deflated 54%)
  adding: best_model_roberta-large_python_tokenizer/ (stored 0%)
  adding: best_model_roberta-large_python_tokenizer/tokenizer_config.json (deflated 80%)
  adding: best_model_roberta-large_python_tokenizer/special_tokens_map.json (deflated 84%)
  adding: best_model_roberta-large_python_tokenizer/merges.txt (deflated 53%)
  adding: best_model_roberta-large_python_tokenizer/vocab.json (deflated 63%)


In [20]:
# Transfer the model to Google drive

from google.colab import drive
drive.mount('/content/drive')

!cp best_model_roberta-large_python.zip "/content/drive/MyDrive/FYP/Revised/Models/roberta-large_Python/"
!cp best_model_roberta-large_python_tokenizer.zip "/content/drive/MyDrive/FYP/Revised/Models/roberta-large_Python_Tokenizer/"

Mounted at /content/drive


# **Load and Test Model**

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

model_folder = '/content/drive/MyDrive/FYP/Revised/Models/roberta-large_Python/'
tokenizer_folder = '/content/drive/MyDrive/FYP/Revised/Models/roberta-large_Python_Tokenizer/'

!unzip '/content/drive/MyDrive/FYP/Revised/Models/roberta-large_Python/best_model_roberta-large_python.zip' -d './'
!unzip '/content/drive/MyDrive/FYP/Revised/Models/roberta-large_Python_Tokenizer/best_model_roberta-large_python_tokenizer.zip' -d './'

Archive:  /content/drive/MyDrive/FYP/Revised/Models/roberta-large_Python/best_model_roberta-large_python.zip
replace ./best_model_roberta-large_python/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_roberta-large_python/pytorch_model.bin  
  inflating: ./best_model_roberta-large_python/training_args.bin  
  inflating: ./best_model_roberta-large_python/config.json  
Archive:  /content/drive/MyDrive/FYP/Revised/Models/roberta-large_Python_Tokenizer/best_model_roberta-large_python_tokenizer.zip
replace ./best_model_roberta-large_python_tokenizer/tokenizer_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_roberta-large_python_tokenizer/tokenizer_config.json  
  inflating: ./best_model_roberta-large_python_tokenizer/special_tokens_map.json  
  inflating: ./best_model_roberta-large_python_tokenizer/merges.txt  
  inflating: ./best_model_roberta-large_python_tokenizer/vocab.json  


In [25]:
best_model_roberta_large_python = 'best_model_roberta-large_python'
best_model_roberta_large_python_tokenizer = 'best_model_roberta-large_python_tokenizer'

In [28]:
# Load the model and tokenizer from the checkpoint
model = RobertaForSequenceClassification.from_pretrained(best_model_roberta_large_python)
tokenizer = RobertaTokenizer.from_pretrained(best_model_roberta_large_python_tokenizer)

text = "for detecting automatically generated fields."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
output

loading configuration file best_model_roberta-large_python/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_s

SequenceClassifierOutput(loss=None, logits=tensor([[-4.8158, -6.0455,  0.8168, -6.3221, -4.5287]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [29]:
logits = output.logits

probabilities = torch.sigmoid(logits)

threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

predicted_classes = (predicted_indices + 1).tolist()

if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class/es: 3
